In [ ]:

# Generate sample data (replace this with your actual time series data)
# Example: X.shape = (num_samples, num_time_steps, num_features)
#          y.shape = (num_samples, output_dim)
X = np.random.rand(100, 10, 2)  # Multivariate input (2 features)
y = np.random.rand(100, 1)

# Define the LSTM model with an added hidden layer
model = Sequential()
model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(LSTM(units=50, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

# Compile the model with RMSE loss
model.compile(optimizer='adam', loss='mean_squared_error')  # RMSE is used as the loss for regression problems

# Train the model
model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)

# Make predictions
predictions = model.predict(X)

# Evaluate the model (optional)
loss = model.evaluate(X, y)
print(f'Root Mean Squared Error on training data: {np.sqrt(loss)}')

# You can now use the trained model for predicting future values


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Generate sample data (replace this with your actual time series data)
# Example: X.shape = (num_samples, num_time_steps, num_features)
#          y.shape = (num_samples, output_dim)
X = torch.rand((100, 10, 2))  # Multivariate input (2 features)
y = torch.rand((100, 2))  # Two output variables

# Define the LSTM model with an added hidden layer
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out, _ = self.lstm1(x)
        out, _ = self.lstm2(out)
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out


# Define the Encoder-Decoder LSTM model
class EncoderDecoderLSTM(nn.Module):
    def __init__(self, n_features, n_steps_in, n_steps_out, hidden_size):
        super(EncoderDecoderLSTM, self).__init__()
        self.encoder = nn.LSTM(n_features, hidden_size, batch_first=True)
        self.repeat_vector = nn.ReplicationPad1d(n_steps_out)
        self.decoder = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, n_features)

    def forward(self, x):
        x, _ = self.encoder(x)
        x = self.repeat_vector(x)
        x, _ = self.decoder(x)
        x = self.output_layer(x)
        return x



# Set hyperparameters
input_size = 2
hidden_size = 50
output_size = 2  # Two output variables
learning_rate = 0.001
epochs = 50
batch_size = 32

# Initialize the model, loss function, and optimizer
model = LSTMModel(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Convert data to PyTorch tensors
X = torch.Tensor(X)
y = torch.Tensor(y)

# Lists to store training and test losses
train_losses = []
test_losses = []

# Training loop
for epoch in range(epochs):
    for i in range(0, X.shape[0] - batch_size + 1, batch_size):
        batch_X = X[i:i+batch_size]
        batch_y = y[i:i+batch_size]

        # Forward pass
        predictions = model(batch_X)

        # Compute the loss
        loss = criterion(predictions, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Training loss
    train_losses.append(loss.item())

    # Validation (test) loss
    with torch.no_grad():
        test_predictions = model(X)
        test_loss = criterion(test_predictions, y)
        test_losses.append(test_loss.item())

    # Print the training and test loss for each epoch
    print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {loss.item()}, Test Loss: {test_loss.item()}')

# Plotting the training and test losses
plt.plot(range(1, epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, epochs + 1), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Test Losses')
plt.show()

# Make predictions
with torch.no_grad():
    predictions = model(X)

# Evaluate the model (optional)
loss = criterion(predictions, y)
print(f'Mean Squared Error on training data: {loss.item()}')

# You can now use the trained model for predicting future values
# 

In [ ]:
def train_backbone(num_epoch, model, X, y, Xval, yval, batchsize, dataaugmentation, criterion, optimizer, filepath, avg, n_save):
  global device

  train_losses = []       # to track the training loss as the model trains
  valid_losses = []       # to track the validation loss as the model trains
  
  avg_train_losses = []   # to track the average training loss per epoch as the model trains
  avg_valid_losses = []   # to track the average validation loss per epoch as the model trains

  dataloader_val = load_data(Xval, yval, batch_size = batchsize, data_augmentation = False)
  dataloader_train= load_data(X, y, batch_size = batchsize, data_augmentation = False)
  #print(dataloader_train.shape)

  for epoch in range(num_epoch):
      #dataloader_train= load_data(X, y, batch_size = batchsize, data_augmentation = dataaugmentation)
    
      try:
        model.train()

        # Train on the current epoch
        for i, data in enumerate(dataloader_train, 0):
            inputs = data["features"].to(device)
            labels = data["labels"].to(device).long()

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)

            loss = criterion(outputs, labels).cuda()
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        # Compute validation loss and accuracy for current epoch
        model.eval()
        with torch.no_grad():  
          for i, data in enumerate(dataloader_val, 0):
              inputs = data["features"].to(device)
              labels = data["labels"].to(device).long()

              outputs = model(inputs)
              # Calcular loss de validación
              loss = criterion(outputs, labels).cuda()
              # record validation loss
              valid_losses.append(loss.item())

        # No se debe guardar checkpoints en cada época (guardarlos cada 50 épocas)
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)

        # limpiar listas para la proxima epoca:
        train_losses = []
        valid_losses = []

        # imprimir resultados por epoca:  
        sys.stdout.write(f"\rÉpoca: {epoch} Training Loss: {train_loss} Validation Loss: {valid_loss}")

        stop = earlyStop(train_loss, valid_loss, avg_train_losses, avg_valid_losses, avg)
        
        if  stop == True:  # caso en que hay que deterner
            # cargar checkpoint, o guardar
            print(" - Early stopping")
            break

        # condición de checkpoint:
        if (epoch % n_save) == 0:
          torch.save(model.state_dict(), filepath)     
      
      # Interrupt Training
      except KeyboardInterrupt:
        print("\nEntrenamiento interrumpido")
        break

  # load the last checkpoint with the best model
  model.load_state_dict(torch.load(filepath))

  print('\nEntrenamiento finalizado')
  return model, avg_train_losses, avg_valid_losses